# Node Classification Version GCN

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import cv2
import argparse
import torch
import tqdm
import random
from torchsummary import summary
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from torchsummary import summary

from models.DAG import DeepAdaptiveGraph
from Dataset import *
from torchvision import transforms
from preprocessing import *
from trainer import experiment
from settings import Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh_DoubleGCN

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False 

args = Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh_DoubleGCN.args

transform_train = transforms.Compose([
    Gamma_2D(),
    Shift_2D(),
#     RandomBrightness(),
    Rotation_2D(),
    RandomSharp(),
    RandomBlur(),
    RandomNoise(),
    ToTensor(),
    ])

transform_test = transforms.Compose([
    ToTensor(),
])

trainset = Dataset_Image_Point_Test(args.data.train, 
                               mode = "train", 
                               transform=transform_train, 
                               train_size=(args.img_height, args.img_width))

validset = Dataset_Image_Point_Test(args.data.valid, 
                               mode = "valid", 
                               transform=transform_test, 
                               train_size=(args.img_height, args.img_width))

train_loader = DataLoader(trainset, batch_size=args.batch_size, shuffle=args.shuffle)
valid_loader = DataLoader(validset, batch_size=1, shuffle=False)

In [ ]:
model = DeepAdaptiveGraph(args, absolute = True, L2 = True, softmax = False, sigmoid = False, clip = False)
# weight = torch.load('./result/Abs_L2_Adj_backbone_hrnet32_Aug_512x512_Tanh/weights/best155_model.pth')
# model.load_state_dict(weight)

logs = experiment(model, train_loader, valid_loader, args)

# 전체 Test데이터 확인

In [ ]:
import pathlib
import copy

img_save_path = os.path.join(args.save_path, 'imgs')
pathlib.Path(img_save_path).mkdir(exist_ok=True, parents=True)

model = DeepAdaptiveGraph(args)
model_dict = torch.load('./result/cnn_learnable_multi_adj_test/weights/best28_model.pth')
model.load_state_dict(model_dict, strict=False)
model.to(args.device)

loss = loss_function()

epoch_val_loss = 0
all_result = []
count=0
with torch.no_grad():
    for batch in test_loader:
        image = torch.tensor(batch[0]).to(args.device).float()
        pre_points = torch.tensor(batch[1]).to(args.device).float()
        move_GT = torch.tensor(batch[2]).to(args.device).float()
        origin_size = torch.tensor(batczh[3])
    
        model.eval()
        pred_points = model(image)
        pred_points = pred_points.view(pred_points.shape[0], -1, 2)
        pred_points.require_grad = False

        val_loss = loss(pred_points, move_GT, origin_size)
        epoch_val_loss += val_loss.item()
        image = image[0].cpu().detach().numpy().transpose((1, 2, 0)).astype(np.int32).copy()
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    
        pre_points[0][7]  = torch.round(pre_points[0][7] + pred_points[0][0])
        pre_points[0][10]  = torch.round(pre_points[0][10] + pred_points[0][1])
        pre_points[0][11] = torch.round(pre_points[0][11] + pred_points[0][2])
        pre_points[0][21]  = torch.round(pre_points[0][21] + pred_points[0][3])

        print(pre_points.shape)
        print(move_GT.shape)
        post_landmark = (move_GT[0]+pre_points[0]).cpu().detach().numpy()
        pre_landmark = pre_points[0].cpu().detach().numpy()
        
        for p in [7,10,11,21]:
            cv2.circle(image, (prediction_landmark[p]).astype('int32'), 3, (0,0,255), -1)
            cv2.circle(image, (pre_landmark[p]).astype('int32'), 3, (0,255,0), -1)
            cv2.circle(image, (post_landmark[p]).astype('int32'), 3, (255,255,0), -1)
        plt.figure(figsize=(20,20))
        plt.imshow(image)
        raise
        #cv2.imwrite(os.path.join(img_save_path, str(count) + '.png'), image)
        #count+=1
        

In [ ]:
sample = next(iter(train_loader))
sample[0].shape

# POST이미지 저장

In [ ]:
import pathlib
import copy
from loss import loss_function

img_save_path = os.path.join(args.save_path, 'imgs')
pathlib.Path(img_save_path).mkdir(exist_ok=True, parents=True)

model = DeepAdaptiveGraph(args)
model_dict = torch.load('./result/cnn_learnable_single_adj_test/weights/best414_model.pth')
model.load_state_dict(model_dict, strict=False)
model.to(args.device)

loss = loss_function()

epoch_val_loss = 0
all_result = []
count=0
with torch.no_grad():
    for batch in test_loader:
        image = torch.tensor(batch[0]).to(args.device).float()
        pre_points = torch.tensor(batch[1]).to(args.device).float()
        move_GT = torch.tensor(batch[2]).to(args.device).float()
        origin_size = torch.tensor(batch[3])
    
        model.eval()
        pred_points = model(image)
        pred_points = pred_points.view(pred_points.shape[0], -1, 2)
        pred_points.require_grad = False

        val_loss = loss(pred_points, move_GT, origin_size)
        epoch_val_loss += val_loss.item()
        image = image[0].cpu().detach().numpy().transpose((1, 2, 0)).astype(np.int32).copy()
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        move_7 = torch.round(pre_points[0][7] + pred_points[0][0]).cpu().detach().numpy()
        move_10 = torch.round(pre_points[0][10] + pred_points[0][1]).cpu().detach().numpy()
        move_11 = torch.round(pre_points[0][11] + pred_points[0][2]).cpu().detach().numpy()
        move_12 = torch.round(pre_points[0][21] + pred_points[0][3]).cpu().detach().numpy()
        four_pridict_points = np.array([move_7,move_10,move_11,move_12])

        gt_move_7 = torch.round(pre_points[0][7] + move_GT[0][0]).cpu().detach().numpy()
        gt_move_10  = torch.round(pre_points[0][10] + move_GT[0][1]).cpu().detach().numpy()
        gt_move_11 = torch.round(pre_points[0][11] + move_GT[0][2]).cpu().detach().numpy()
        gt_move_12  = torch.round(pre_points[0][21] + move_GT[0][3]).cpu().detach().numpy()
        four_gt_points = np.array([gt_move_7,gt_move_10,gt_move_11,gt_move_12])
        
        pre_7 = torch.round(pre_points[0][7]).cpu().detach().numpy()
        pre_10  = torch.round(pre_points[0][10]).cpu().detach().numpy()
        pre_11 = torch.round(pre_points[0][11]).cpu().detach().numpy()
        pre_12  = torch.round(pre_points[0][21]).cpu().detach().numpy()
        pre_points = np.array([pre_7,pre_10,pre_11,pre_12])
    
       
        
        for p in range(4):
            
            cv2.circle(image, (four_pridict_points[p]).astype('int32'), 3, (255,0,0), -1)
            cv2.circle(image, (pre_points[p]).astype('int32'), 3, (255,255,0), -1)
            cv2.circle(image, (four_gt_points[p]).astype('int32'), 3, (0,255,0), -1)
        plt.figure(figsize=(20,20))
        plt.imshow(image)
        plt.show()
        #raise
        #cv2.imwrite(os.path.join(img_save_path, str(count) + '.png'), image)
        #count+=1